<a href="https://colab.research.google.com/github/martin-ibarra0179/topicos_industria_1_practica_2/blob/develop/Topicos_industria_1_Practica_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Practica 2

## Modulos

In [ ]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from scipy.stats import levy_stable

## Lévy distribution - N Curvas

In [ ]:
# parametros iniciales
miu = 3.0
levy_exponent = 0.2
beta = 0
resolution = 200
aux_domain = np.linspace(miu-5, miu+5, resolution)

figure_levy_pdf = go.Figure()
levy_pdf = np.array([levy_stable.pdf(i, alpha=levy_exponent, beta=beta, loc=miu) for i in aux_domain])
figure_levy_pdf.add_trace(go.Scatter(
    x=aux_domain,
    y=levy_pdf,
    marker=dict(size=2)
))
figure_levy_pdf.show()